In [93]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import string
import os
import networkx as nx
import codecs
from os import path
from bs4 import BeautifulSoup
from string import digits
from nltk.stem.snowball import FrenchStemmer
import nltk
nltk.download('wordnet')
# spacy for lemmatization
import spacy
!python -m spacy download fr


[nltk_data] Downloading package wordnet to /Users/siwar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     |████████████████████████████████| 14.7MB 2.0MB/s eta 0:00:01
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp37-none-any.whl size=14727027 sha256=8a6f399f806badcd48b5166c31c084f6b7da4e581280f9c8f4409d809acd1002
  Stored in directory: /private/var/folders/bl/b25md3vx7_57j7sltyf3z8140000gn/T/pip-ephem-wheel-cache-l53bur33/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/opt/anaconda3/lib/python3.7/site-packages/fr_core_news_sm -->
/opt/anaconda3/lib/python3.7/site-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


## 1.Load Data

In [12]:
ls ..

cross_val.py         models/              utils/
data/                submissions.csv      write_submission.py


In [17]:
def get_data_full_modified():
    with open("../data/train.csv", 'r') as f:
        train_data = f.read().splitlines()
    with open("../data/test.csv", 'r') as f:
        test_hosts = f.read().splitlines()

    train_hosts = list()
    y_train = list()
    for row in train_data:
        host, label = row.split(",")
        train_hosts.append(host)
        y_train.append(label.lower())

    # Text data
    # Load the textual content of a set of webpages for each host into the dictionary "data".
    # The encoding parameter is required since the majority of our data is french.
    text = dict()
    filenames = os.listdir('../data/text')
    
    for filename in filenames:
        try:
            with codecs.open(path.join('../data/text/', filename), encoding='utf-8') as f:
                text[filename] = f.read().replace("\n", "").lower()
        except:
            with codecs.open(path.join('../data/text/', filename), encoding='latin-1') as f:
                text[filename] = f.read().replace("\n", "").lower()
    
    X_train = list()
    for host in train_hosts:
        if host in text:
            X_train.append([host, text[host]])
        else:
            X_train.append([host, ''])
    # Get textual content of web hosts of the test set
    X_test = list()
    for host in test_hosts:
        if host in text:
            X_test.append([host, text[host]])
        else:
            X_test.append([host, ''])
    return X_train, y_train, X_test, test_hosts

In [65]:
X_train, y_train, X_test, test_hosts = get_data_full_modified() 

In [66]:
data_train = pd.DataFrame({'text': X_train,'category': y_train,})
data_train.head()

,text,category
0,"[9032, #polepharma » flux polepharma » flux...",health/medical
1,"[5346, 301 moved p...",entertainment
2,"[18778, (button) fermer en poursuivant vo...",entertainment
3,"[11939, #hal (button) toggle navigation ...",education/research
4,"[17502, user-agent: * disallow: disallow: /...",tech/science


In [67]:
data_train[['train_host','text']] = pd.DataFrame(data_train.text.values.tolist(), index= data_train.index)
data_train.set_index('train_host', inplace=True)
data_train.head()

,text,category
train_host,,
9032,#polepharma » flux polepharma » flux des co...,health/medical
5346,301 moved permanen...,entertainment
18778,(button) fermer en poursuivant votre navi...,entertainment
11939,#hal (button) toggle navigation ccsd ...,education/research
17502,user-agent: * disallow: disallow: /publishe...,tech/science


In [68]:
data_train.shape

(2125, 2)

In [69]:
print(data_train['text']['9032'])

   #polepharma » flux polepharma » flux des commentaires alternate   alternate   polepharma   polepharma ﻿ recherche_________ submit   (button) toggle navigation   polepharma     * polepharma          + qui sommes-nous ?          + organigramme          + les chiffres clés          + un réseau     * territoires          + région centre-val de loire          + région normandie          + conseil départemental d’eure&loir          + chartres metropole          + metropole de rouen          + agglo du pays de dreux          + agglo evreux portes de normandie          + agglo seine&eure          + agglo de tours plus     * performance industrielle          + excellence opérationnelle          + maintenance          + packaging          + supply chain          + usine du futur     * talents     * relations publiques     * innovation          + biomédicaments          + fédérer          + collaborer          + promouvoir   adhérer à polepharma   espace adhérent   rechercher un adhérent   je 

In [70]:
data_test = pd.DataFrame({'text': X_test,})
data_test.head()

,text
0,"[27997, iframe: //www.googletagmanager.com/..."
1,"[9316, iframe: https://www.googletagmanager..."
2,"[27045, #toutes les astuces beauté ⋅ astuce..."
3,"[19805, [logodefault.jpg] [etab_juvisy-su..."
4,"[26580, #l'école de demain » flux l'école d..."


In [71]:
data_test[['test_host','text']] = pd.DataFrame(data_test.text.values.tolist(), index= data_test.index)
data_test.set_index('test_host', inplace=True)
data_test.head()

,text
test_host,
27997,iframe: //www.googletagmanager.com/ns.html?...
9316,iframe: https://www.googletagmanager.com/ns...
27045,#toutes les astuces beauté ⋅ astuces beauté...
19805,[logodefault.jpg] [etab_juvisy-sur-orge.p...
26580,#l'école de demain » flux l'école de demain...


In [72]:
data_test.shape

(560, 1)

In [73]:
labels = ['business/finance', 'education/research', 'entertainment', 'health/medical',
          'news/press', 'politics/government/law', 'sports', 'tech/science']

## 2.Preprocessing

In [74]:
### Remove html tags and uris from contents
uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

In [75]:
# This could take a while

data_train["text"] = data_train["text"].map(stripTagsAndUris)

In [76]:
data_train["text"]['9032']

'   #polepharma » flux polepharma » flux des commentaires alternate   alternate   polepharma   polepharma \ufeff recherche_________ submit   (button) toggle navigation   polepharma     * polepharma          + qui sommes-nous ?          + organigramme          + les chiffres clés          + un réseau     * territoires          + région centre-val de loire          + région normandie          + conseil départemental d’eure&loir          + chartres metropole          + metropole de rouen          + agglo du pays de dreux          + agglo evreux portes de normandie          + agglo seine&eure          + agglo de tours plus     * performance industrielle          + excellence opérationnelle          + maintenance          + packaging          + supply chain          + usine du futur     * talents     * relations publiques     * innovation          + biomédicaments          + fédérer          + collaborer          + promouvoir   adhérer à polepharma   espace adhérent   rechercher un adhérent

In [84]:
def clean(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    #x = re.sub(r'[^\x00-\x7f]',r' ',x)
    x = re.sub(r'ô','o',x)
    x = re.sub(r'[éêè]','e',x)
    x = re.sub(r'([^a-zéèûôàêô])',' ',x)
    x = re.sub('\n', '',x)
    x = re.sub("(\s+.\s+)", ' ', x)#remove any single charecters hanging between 2 spaces

    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [85]:
data_train["text"] = data_train["text"].map(clean)

In [86]:
#Removing stopwords 
from nltk.corpus import stopwords
import numpy as np
import pickle

stops = set(stopwords.words('french'))
def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if word not in stops]
    return " ".join(filtered_words)

def removedigits(x):
    remove_digits = str.maketrans('', '', digits)
    res = x.translate(remove_digits)
    return res

In [87]:
data_train["text"] = data_train["text"].map(removeStopwords)
data_train["text"] = data_train["text"].map(removedigits)

In [88]:
data_train["text"]['9032']

'polepharma flux polepharma flux commentaires alternate alternate polepharma polepharma recherche submit button toggle navigation polepharma polepharma organigramme chiffres cles reseau territoires region centre val loire region normandie conseil departemental eure loir chartres metropole metropole rouen agglo pays dreux agglo evreux portes normandie agglo seine eure agglo tours plus performance industrielle excellence operationnelle maintenance packaging supply chain usine futur talents relations publiques innovation biomedicaments federer collaborer promouvoir adherer polepharma espace adherent rechercher adherent decouvre metiers filiere pharmaceutique polepharma mediatheque espace presse polepharma polepharma recherche submit polepharma polepharma polepharma organigramme chiffres cles reseau territoires region centre val loire region normandie conseil departemental eure loir chartres metropole metropole rouen agglo pays dreux agglo evreux portes normandie agglo seine eure agglo tou

In [126]:
# a second round of cleaning

def lemmetize(x):
    
    nlp = spacy.load('fr', disable=['parser', 'ner'])
  
    doc = nlp(x)
            
    lemmas = ' '.join(token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN','VERB', 'ADJ'])
    
    return lemmas

In [127]:
s = "plus performance industrielle excellence operationnelle maintenance "
lemmetize(s)

'performance industriel excellence operationnell maintenance'

In [25]:
data_train["text"] = data_train["text"].map(lemmetize)

In [26]:
data_train["text"]['9032']

'polepharma flux commentaire alternate alternat polepharma recherche submit button toggle navigation polepharma polepharma chiffre cles territoire region centre val loir region normandie conseil eur chartre agglo pays dreux agglo evreux porte normandie agglo seine eur agglo tour performance industriel excellence operationnell maintenance packaging supply chain usine futur talent relation public innovation biomedicament federer collaborer promouvoir adherer polepharma rechercher metier filiere pharmaceutique polepharma mediathequ espace presse polepharma recherche polepharma polepharma chiffre cles territoire region centre val loir region normandie conseil eur chartre agglo pays dreux agglo evreux porte normandie agglo seine eur agglo tour performance industriel excellence operationnell maintenance packaging supply chain usine futur talent relation public innovation biomedicament federer collaborer promouvoir national emergence projet collaboratif innovationlancemer national emergence p

In [31]:
data_train.head()

,text,category
train_host,,
9032,polepharma flux commentaire alternate alternat...,health/medical
5346,moved permanently nginx moved permanently ngin...,entertainment
18778,button fermer navigation modifier parametre co...,entertainment
11939,button toggle navigation hal hal halsh tel med...,education/research
17502,user disallow publisher user yandex crawl tele...,tech/science


In [40]:
data_train.to_csv('clean_data.csv')